--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Aysu Avcı](https://github.com/aysuavci)

# Replication of Zimmermann (2020) <a class="tocSkip">   
---

This notebook contains the replication of the following study:

> [Zimmermann, F. (2020). The Dynamics of Motivated Beliefs. American Economic Review, 110(2), 337–363](https://www.aeaweb.org/articles?id=10.1257/aer.20180728).


##### Notes:

* I try to remain true to the original naming of the variables, figures and tables as in the study of Zimmermann (2020), so that viewing and comparison would be more convenient.

* For the best viewing, the responsitory can be downloaded from [here](https://github.com/OpenSourceEconomics/ose-data-science-course-project-aysuavci).

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api
import matplotlib as plt
from IPython.display import HTML

In [2]:
from auxiliary.example_project_auxiliary_predictions import *
from auxiliary.example_project_auxiliary_plots import *
from auxiliary.example_project_auxiliary_tables import *

---
# 1.Introduction 
---

The study of Zimmermann(2020) is a lab experiment aiming to examine how motivated beliefs held by individuals continues to be sustained after receiving positive or negative feedback.
The study can be divided into 3 parts:

1. The first part of the study-named “**Motivated Belief Dynamics**” in the paper- is the main study for investigating the causality between different types of feedback (positive or negative) received and reconstruction of belief patterns depending on the elicitation time (directly or 1 month) of the belief after the experiment. 
2. The second part of the study-named “**The Role of Memory**” in the paper- investigates the asymmetry in the accuracy of feedback recall and also recall of IQ test in general that they solve as a part of the experiment. 
3. The final part of the study-named “**The Trade-Off between Motivated and Accurate Belief**” in the paper- that questions whether incentivizing for recall accuracy could mitigate the motivated reasoning that participants employ.

In this project notebook, the first and the main part (and the last parts) of the study will be replicated; where Zimmermann (2020) used also difference-in-difference models for their estimations. In the next section (Section 2), I will explain only relevant parts of the experimental design of the study and introduce all treatment groups and variables that are going to be mentioned in this notebook. In Section 3, I will explain the estimation strategy employed by Zimmermann (2020) and the models being used….(going to change)


---
# 2. Experimental Design
---
1. Subjects completed an IQ test where they need to solve 10 Raven matrices.
2. Subjects were randomly assigned to groups of 10.
3. Subjects were asked to estimate their belief about the likelihood of being ranked in the upper half of their group in terms of a percantage point. Full probability distributions for every possible rank is also elicited.
4. Subjects were given noisy but true information about their rank by randomly selecting 3 participants from their group and telling them whether they were rank higher or lower than these 3 participants. If a subject is told that they ranked higher than 2 or 3 participants in their group, the subject considered to be in the positive feedback group. They considered being in the negative feedback group if otherwhise. 
5. After receiving the feedbacks, subjects are divided into two treatment groups randomly: `condifence_direct` and `confidence_1monthlater`. 
6. In the `confidence_1monthlater` treatment, subjects are asked to elicit their beliefs 1 month later; and in the `condifence_direct` treatment, group subjects are asked to elicit their beliefs after the feedback. `condifence_direct`has also two subgroups: `condifence_direct_immediate` and `condifence_direct_15minutelater`.


**Main Variables**

|Treatment groups|Feedback types|Outcomes|Other variables|
|---|---|---|---|
|`confidence_1monthlater`|Positive|Belief Adjustment|rank|
|`condifence_direct_immediate`|Negative||
|`condifence_direct_15minutelater`|||


---
# 3. Identification
--- 

The first part of the experiment is dedicated to investigating the causality between the effect of feedback and belief adjustments of participants that are in two different treatment groups that vary depending on time: _CondifenceDirect_ and _Condifence1month_. The causality is established by two key components in the experiment. First of all, Zimmermann(2020) elicit peoples prior beliefs about their probability of ranking in the upper half of the group and then they elicited again after participants received the feedback. So, for each participant, they had a clear measure of belief updating which is possibly caused by the feedback and in line with the type of feedback. The second and the most important component of the experiment, that ensured causal identification, is the noisy feedback component. In this experiment rather than randomly assigning a subject into a negative or positive feedback group directly, Zimmermann(2020) randomly choose 3 other participants within the subject’s group to compare their rank and provide them with 3 comparisons. In this way, subjects indeed received true feedback but a noisy one. So, it is possible for two subjects with the same rank to receive different types of feedback. By adopting such an experimental design, Zimmermann(2020) ensured that potential asymmetries in the belief dynamics cannot be explained by the individual difference between participants. 

In addition to investigating the causal relationship between belief dynamics and feedback, another goal of the experiment is to observe the effect of time on this relationship. For that, Zimmermann(2020) assigned participants randomly to two groups: Confidence1month and CondifenceDirect. ConfidenceDirect is also divided into two subgroups(immediate and 15 minutes later belief elicitation) to investigate the possible short-term dynamics in belief adjustment, however, this replication will not cover that part of the study. 

--CAUSAL GRAPH 

---
# 4. Empirical Strategy
---


In the study, the belief adjustment is defined as the difference between the elicited belief after the feedback and the belief before the feedback, formally

\begin{equation}
Pr(upperhalf)^{post}_{i} - Pr(upperhalf)^{prior}_{i}
\end{equation}

where $Pr(upperhalf)_{i}$ represents the subject ${i}$'s given probability of being in the upperhalf of the group.

To compare belief adjustments between positive and negative feedbacks, a normalized version of the belief adjustment norm is also formed as below:


$$
beliefadjustmentnorm_i = \left\{
    \begin{array}\\
        Pr(upperhalf)^{post}_{i} - Pr(upperhalf)^{prior}_{i} & \mbox{if } \ feedback \ positive \\
        (-1) x (Pr(upperhalf)^{post}_{i} - Pr(upperhalf)^{prior}_{i}) & \mbox{if } \ feedback \ negative \\
    \end{array}
\right.
$$


The difference-in-difference models, formally in the form of 

\begin{equation}
beliefadjustmentnorm_i = \alpha + \beta feedback_i + \gamma T_i + \delta I_i + X_i \gamma + \epsilon_i
\end{equation}

are estimated for the purpose of establishing dynamic belief patterns. $feedback_i$ is the dummy variable for feedback and $T_i$ is the type of treatment. $I_i$ is for the interation term that is a generated dummy for the interest group, interest group being, subjects who are in the `confidence_1monthlater`and received a negative feedback; parameter $\delta$ is therefore captures the belief dynamics. In order to form a causal inference, Zimmermann (2020) control for rank and IQ test scores; these variables are captured by $X_i$, set of control variables. 

As another control variable, Bayesian belief adjustments are also used for some specifications. Zimmermann (2020) calculated Bayesian predictions, using individuals full prior probability distribution and the feedback type that they received. The Bayesian belief adjustment is then, produced in the same way as the belief adjustment

\begin{equation}
Pr(upperhalf)^{postBayes}_{i} - Pr(upperhalf)^{prior}_{i}
\end{equation}

where $Pr(upperhalf)^{postBayes}_{i}$ is the calculated Bayesian prediction for the subject $i$.

Another specification within the study is adding the rank fixed effects interacted with treatment. In this way, Zimmermann (2020) was able to control the differentiation of possible different characteristics due to ranking between treatment groups.

Zimmermann (2020) choose to perform ordinary least square (OLS) estimates with type HC1 heteroscedasticity robust covariance. Most of the main results are produced with this linear regression and I will use this type of regression in the table replications. In the study, Zimmermann (2020) perform 8 OLS estimations(see in Table 1):<br>
(1) Regressing $y_i$ (normalized belief adjustment) on $T_i$ (treatment type) in case of positive feedback<br>
(2) Same as (1) but controling for $X_i$ (rank and normalized bayesian belief adjustment)<br>
(3) Regressing $y_i$ (normalized belief adjustment) on $T_i$ (treatment type) in case of negative feedback<br>
(4) Same as (3) but controling for $X_i$ (rank and normalized bayesian belief adjustment)<br>
(5) Regressing $y_i$ (normalized belief adjustment) on $T_i$ (treatment type), $feedback_i$ (feedback type) and $I_i$ (interaction term indicating whether individual is in the interest group or not)<br>
(6) Same as (5) but controling for $X_i$ (rank and normalized bayesian belief adjustment)<br>
(7) Same as (5) but including rank fixed effects with their interaction with treatment<br>
(8) Same as (7) but controling for $X_i$ (normalized bayesian belief adjustment)<br>

---
# 5. Replication of Zimmermann (2020)
---

## 5.1. Data & Descriptive Statistics

In [5]:
df = pd.read_stata('data/data.dta')

In [7]:
#Adjustments for better viewing
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 7)
pd.set_option("display.max.rows", 30)

In [8]:
#Creating variable treatgroup 
tg_conditions = [
    (df['treatment'] == 'belief_announcement'),
    (df['treatment'] == 'confidence_1monthlater'),
    (df['treatment'] == 'confidence_direct_15minuteslater'),
    (df['treatment'] == 'confidence_direct_immediate'),
    (df['treatment'] == 'memory'),
    (df['treatment'] == 'memory_high'),
    (df['treatment'] == 'nofeedback'),
    (df['treatment'] == 'tournament_announcement'),
]
tg_values = [1, 2, 3, 4, 5, 6, 7, 8]
df['treatgroup'] = np.select(tg_conditions, tg_values)

#### 5.1.1 Creation of variables needed for estimations

In [7]:
#Generating bayesian predictions
df["post_1"] = df["prior_1"]/100

df.loc[df['pos_comparisons'] == 0, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 1, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_1'] = df["prior_1"]/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_2"] = df["prior_2"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_2'] = (df["prior_2"]*(1/9)*(1/9)*(1/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_2'] = (df["prior_2"]*(24/9)*(1/9)*(1/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_2'] = (df["prior_2"]*(24/9)*(8/9)*(1/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_2'] = (df["prior_2"]*(8/9)*(8/9)*(8/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_3"] = df["prior_3"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_3'] = (df["prior_3"]*(2/9)*(2/9)*(2/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_3'] = (df["prior_3"]*(21/9)*(2/9)*(2/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_3'] = (df["prior_3"]*(21/9)*(7/9)*(2/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_3'] = (df["prior_3"]*(7/9)*(7/9)*(7/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_4"] = df["prior_4"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_4'] = (df["prior_4"]*(3/9)*(3/9)*(3/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_4'] = (df["prior_4"]*(18/9)*(3/9)*(3/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_4'] = (df["prior_4"]*(18/9)*(6/9)*(3/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_4'] = (df["prior_4"]*(6/9)*(6/9)*(6/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_5"] = df["prior_5"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_5'] = (df["prior_5"]*(4/9)*(4/9)*(4/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_5'] = (df["prior_5"]*(15/9)*(4/9)*(4/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_5'] = (df["prior_5"]*(15/9)*(5/9)*(4/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_5'] = (df["prior_5"]*(5/9)*(5/9)*(5/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_6"] = df["prior_6"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_6'] = (df["prior_6"]*(5/9)*(5/9)*(5/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_6'] = (df["prior_6"]*(12/9)*(5/9)*(5/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_6'] = (df["prior_6"]*(12/9)*(4/9)*(5/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_6'] = (df["prior_6"]*(4/9)*(4/9)*(4/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_7"] = df["prior_7"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_7'] = (df["prior_7"]*(6/9)*(6/9)*(6/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_7'] = (df["prior_7"]*(9/9)*(6/9)*(6/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_7'] = (df["prior_7"]*(9/9)*(3/9)*(6/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_7'] = (df["prior_7"]*(3/9)*(3/9)*(3/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_8"] = df["prior_8"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_8'] = (df["prior_8"]*(7/9)*(7/9)*(7/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_8'] = (df["prior_8"]*(6/9)*(7/9)*(7/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_8'] = (df["prior_8"]*(6/9)*(2/9)*(7/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_8'] = (df["prior_8"]*(2/9)*(2/9)*(2/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_9"] = df["prior_9"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_9'] = (df["prior_9"]*(8/9)*(8/9)*(8/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_9'] = (df["prior_9"]*(3/9)*(8/9)*(8/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_9'] = (df["prior_9"]*(3/9)*(1/9)*(8/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_9'] = (df["prior_9"]*(1/9)*(1/9)*(1/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_10"] = df["prior_10"]/100

df.loc[df['pos_comparisons'] == 0, 'post_10'] = df["prior_10"]/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9) + df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_10'] = 0

In [9]:
#Generate posterior_median_bayes

df["posterior_median_bayes"] = (df["post_1"] + df["post_2"] + df["post_3"] + df["post_4"] + df["post_5"])*100

Throughout this study feedback groups will be addressed using the dummy variable `dummynews_goodbad`. As is mentioned above, if a subject received more than 2 positive comparisons they are considered to be in the positive feedback group; if else in the negative feedback group.

In [57]:
#Creating dummy variable for good/bad news (if good = 0, if bad = 1)
df["dummynews_goodbad"] = np.nan
df.loc[(df['pos_comparisons'] == 2) | (df['pos_comparisons'] == 3), 'dummynews_goodbad'] = 0
df.loc[(df['pos_comparisons'] == 0) | (df['pos_comparisons'] == 1), 'dummynews_goodbad'] = 1

In [ ]:
#### 5.2 Creation of variables needed for estimations

In [59]:
#Creating data frames depending on `dummynews_goodbad`
news_good_ba = df[df["dummynews_goodbad"] == 0]
news_bad_ba = df[df["dummynews_goodbad"] == 1]